## Compile PI-detector

1. Pull `ubuntu:latest` image from docker and run.

2. Install necessary dependencies

   ```bash
   apt-get update && apt-get install -y --no-install-recommends apt-utils
   
   apt-get install -y --no-install-recommends \
     clang-18 clang-tools-18 clang-18-doc libclang-common-18-dev \
     libclang-18-dev libclang1-18 clang-format-18 python3-clang-18 \
     clangd-18 clang-tidy-18 \
     python3 python3-setuptools python3-pip \
     cmake git ca-certificates build-essential
   ```

3. Clone our project to `/` directory.

4. Make install

   ```bash
   // pwd: /pi-detector
   mkdir build && cd build
   cmake ..
   make install
   ```

5. Build GSL

   ```bash
   // pwd: /pi-detector
   cd externel/gsl
   bash compile_gsl_sf.sh
   ```

6. Make pi-detector

   ```bash
   // pwd: /pi-detector
   cd build
   make detector
   ```



## Generate the Full `Single Point`

```bash
// pwd: /script
python3 get_inputs.py gsl 0 0 /pi-detector/data/atom/in_sp_full.json  // GSL
python3 get_inputs.py hsed 0 0 /pi-detector/data/hsed/in_sp_full.json // HSED
```

### Calculate TP/TN/FP/FN

#### Test GSL in PI-detector

**Get the GSL result**

```bash
// pwd: /pi-detector/build/bin
./detector gsl input /pi-detector/data/atom/in_sp_full.json /pi-detector/data/atom/out_sp_full.json --relerr
```

**Get the Mpmath result**
```bash
// pwd: /pi-detector/script
python evaluation.py -l atom -d /pi-detector/data/atom/out_sp_full.json --relerr
```

**Calculate TP/TN/FP/FN**
```bash
// pwd: /pi-detector/script
python tpfn-detector.py /pi-detector/data/atom/out_sp_full.json
```

#### Test HSED in PI-detector

**Get the HSED result**

```bash
// pwd: /pi-detector/build/bin
./detector hsed input /pi-detector/data/hsed/in_sp_full.json /pi-detector/data/hsed/out_sp_full.json
```

**Get the MPFR result**

Deploy [HSED](https://github.com/zuoyanzhang/HSED) and get the MPFR results for each input in `/pi-detector/data/hsed/out_sp_full.json`.


**Calculate TP/TN/FP/FN**
```bash
// pwd: /pi-detector/script
python tpfn-detector.py /pi-detector/data/hsed/out_sp_full.json
```


#### Test GSL in FPCC

**Get the GSL result**

```bash
// pwd: /pi-detector/build/bin
./detector gsl input /pi-detector/data/atom/in_sp_full.json /pi-detector/data/fpcc/out_sp_full.json
```

**Get the FPCC fitness**

Deploy [FPCC](https://github.com/DataReportRe/FPCC) and get the FPCC `fitness` for each input in `/pi-detector/data/hsed/out_sp_full.json`.

**Calculate TP/TN/FP/FN**
```bash
// pwd: /pi-detector/script
python tpfn-fpcc.py /pi-detector/data/fpcc/out_sp_full.json
```

#### Test HSED in FPCC

Append [HSED function](/pi-detector/external/hsed/hsed.c) to FPCC to calculate the FPCC fitness.
Then use the following cmd to calculate the TPFN
```bash
// pwd: /pi-detector/script
python tpfn-fpcc.py `/path/to/your/file`
```

----------------------------

## Generate 172 (ATOMU) + 2 (HSED) Inputs (single point) with sig. err.

```bash
python3 get_inputs.py gsl 0 0 /pi-detector/data/atom/in_sigerr_sp_full.json --sigerr  // ATOMU
python3 get_inputs.py hsed 0 0 /pi-detector/data/hsed/in_sigerr_sp_full.json --sigerr // HSED
```

Use above method to get `TABLE III: Errors calculated by our approach, high-precision approach, and FPCC.`


## Generate 172 (ATOMU) + 2 (HSED) Inputs (expand) with sig. err.

```bash
python3 get_inputs.py gsl 1000 10 /pi-detector/data/atom/in_sigerr_expand.json --sigerr  // ATOMU
python3 get_inputs.py hsed 1000 1 /pi-detector/data/hsed/in_sigerr_expand.json --sigerr // HSED
```

Use above method to get all results. 

Then use

```python
import pymannkendall as mk
result = mk.original_test(errors) # The error array.
```

to get `TABLE IV: Runtime of floating-point error calculations by using PI-detector, FPCC, and high-precision approach`

## Matrix

See [matrix folder](/pi-detector/matrix).

Install required libraries, and put built [gsl lib](/pi-detector/external/gsl/build) inside.

Use `Makefile` to generate executables.

Use `generator` to generate 3000 matrices with 1000x1000.

Use `mplapack`, `gsl_solver` and `gsl_solver_i` to generate the high-precision result, GSL result and perturbed GSL result. 

Finally, use `compare.py` to compare three results.